In [1]:
import torch
import json
from tqdm import tqdm
from itertools import product as p
import numpy as np

In [3]:
class A:
    pass
self = A()

In [4]:
self.num_neurons_d = {} # {fname, int}
self.representations_d = {} # {fname, tensor}
f1, f2, f3 = "foo", "bar", "baz"
representation_files = [f1, f2, f3]

In [5]:
# initialize `num_neurons_d`, `representations_d` with fake data
n1, n2, n3 = 100, 80, 70
nword = 1000
t1 = torch.randn(nword, n1)
t2 = torch.randn(nword, n2)
t3 = torch.randn(nword, n3)
self.num_neurons_d[f1] = n1
self.num_neurons_d[f2] = n2
self.num_neurons_d[f3] = n3
self.representations_d[f1] = t1
self.representations_d[f2] = t2
self.representations_d[f3] = t3

In [6]:
# extra params
self.limit = None

# gram_rbf

In [7]:
# input
X = torch.randn(10, 5)
threshold = 1.0

In [8]:
dot_products = torch.mm(X, X.t())
sq_norms = torch.diag(dot_products)

In [9]:
sq_distances = -2 * dot_products + sq_norms[:, None] + sq_norms[None, :]

In [10]:
Y = torch.empty(10, 10)
for i, j in p(range(10), range(10)):
    Y[i, j] = torch.norm(X[i] - X[j])
Y**2 - sq_distances # all small

tensor([[ 0.0000e+00,  0.0000e+00,  4.7684e-07, -9.5367e-07,  4.7684e-07,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -9.5367e-07],
        [ 0.0000e+00,  0.0000e+00,  1.9073e-06,  9.5367e-07,  0.0000e+00,
         -9.5367e-07,  1.9073e-06,  0.0000e+00, -1.9073e-06, -9.5367e-07],
        [ 4.7684e-07,  1.9073e-06,  0.0000e+00,  0.0000e+00, -4.7684e-07,
          0.0000e+00, -4.7684e-07,  2.3842e-07,  7.1526e-07,  0.0000e+00],
        [-9.5367e-07,  9.5367e-07,  0.0000e+00,  0.0000e+00,  4.7684e-07,
         -1.1921e-07,  0.0000e+00, -1.1921e-07,  0.0000e+00,  4.7684e-07],
        [ 4.7684e-07,  0.0000e+00, -4.7684e-07,  4.7684e-07,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-9.5367e-07, -9.5367e-07,  4.7684e-07, -1.1921e-07,  0.0000e+00,
          0.0000e+00, -4.7684e-07,  0.0000e+00,  9.5367e-07,  0.0000e+00],
        [-9.5367e-07,  1.9073e-06, -4.7684e-07,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+0

In [11]:
sq_median_distance = torch.median(sq_distances)
torch.exp(-sq_distances / (2 * threshold ** 2 * sq_median_distance))

tensor([[1.0000, 0.7507, 0.5388, 0.4575, 0.6136, 0.3495, 0.4465, 0.3753, 0.4071,
         0.4197],
        [0.7507, 1.0000, 0.5407, 0.4774, 0.4748, 0.4795, 0.2608, 0.3665, 0.3235,
         0.2975],
        [0.5388, 0.5407, 1.0000, 0.8286, 0.6065, 0.5740, 0.5129, 0.7181, 0.8231,
         0.7575],
        [0.4575, 0.4774, 0.8286, 1.0000, 0.6817, 0.8463, 0.7088, 0.8859, 0.7702,
         0.5360],
        [0.6136, 0.4748, 0.6065, 0.6817, 1.0000, 0.6268, 0.6628, 0.4440, 0.5301,
         0.6351],
        [0.3495, 0.4795, 0.5740, 0.8463, 0.6268, 1.0000, 0.5055, 0.6384, 0.4547,
         0.3281],
        [0.4465, 0.2608, 0.5129, 0.7088, 0.6628, 0.5055, 1.0000, 0.6822, 0.6241,
         0.4274],
        [0.3753, 0.3665, 0.7181, 0.8859, 0.4440, 0.6384, 0.6822, 1.0000, 0.7662,
         0.3774],
        [0.4071, 0.3235, 0.8231, 0.7702, 0.5301, 0.4547, 0.6241, 0.7662, 1.0000,
         0.6572],
        [0.4197, 0.2975, 0.7575, 0.5360, 0.6351, 0.3281, 0.4274, 0.3774, 0.6572,
         1.0000]])

In [ ]:
# full
def gram_rbf(X, threshold=1.0):
    dot_products = torch.mm(X, X.t())
    sq_norms = torch.diag(dot_products)
    sq_distances = -2 * dot_products + sq_norms[:, None] + sq_norms[None, :]
    sq_median_distance = torch.median(sq_distances)
    return torch.exp(-sq_distances / (2 * threshold ** 2 * sq_median_distance))

In [12]:
def center_gram(G):
    means = G.mean(0, keepdim=False)
    means -= means.mean() / 2
    return G - means[None, :] - means[:, None]

# compute_correlations

In [ ]:
# Set `limit`
n_words = next(iter(self.representations_d.values())).size()[0]
if type(self.limit) == float:
    limit = int(n_words * self.limit)
elif type(self.limit) == int:
    limit = self.limit
else:
    limit = self.limit

In [15]:
# Set `self.similarities`
# {network: {other: rbfcka_similarity}}
self.similarities = {network: {} for network in self.representations_d}

In [16]:
# loopvar
network = f1
other_network = f2

In [17]:
# if network == other_network:
#     continue

# if other_network in self.similarities[network]: 
#     continue

In [18]:
# TO DO: random subset of data using limit?
Gx = center_gram(gram_rbf(self.representations_d[network][:limit]))
Gy = center_gram(gram_rbf(self.representations_d[other_network][:limit]))

In [19]:
scaled_hsic = torch.dot(Gx.view(-1), Gy.view(-1)).item()

In [20]:
norm_gx = torch.norm(Gx, p="fro").item()
norm_gy = torch.norm(Gy, p="fro").item()

In [21]:
sim = scaled_hsic / (norm_gx*norm_gy)
self.similarities[network][other_network] = sim
self.similarities[other_network][network] = sim

In [22]:
# full loop
for network, other_network in tqdm(p(self.representations_d,
                                             self.representations_d),
                                           desc='rbfcka',
                                           total=len(self.representations_d)**2):

    if network == other_network:
        continue

    if other_network in self.similarities[network]: 
        continue

    # TO DO: random subset of data using limit?
    Gx = center_gram(gram_rbf(self.representations_d[network][:limit]))
    Gy = center_gram(gram_rbf(self.representations_d[other_network][:limit]))

    scaled_hsic = torch.dot(Gx.view(-1), Gy.view(-1)).item()

    norm_gx = torch.norm(Gx, p="fro").item()
    norm_gy = torch.norm(Gy, p="fro").item()

    sim = scaled_hsic / (norm_gx*norm_gy)
    self.similarities[network][other_network] = sim
    self.similarities[other_network][network] = sim

rbfcka: 100%|█| 9/9 [00:00<00:00, 111.55it/s]


# Full functions

In [31]:
# full
def gram_rbf(X, threshold=1.0):
    dot_products = torch.mm(X, X.t())
    sq_norms = torch.diag(dot_products)
    sq_distances = -2 * dot_products + sq_norms[:, None] + sq_norms[None, :]
    sq_median_distance = torch.median(sq_distances)
    return torch.exp(-sq_distances / (2 * threshold ** 2 * sq_median_distance))

In [32]:
def center_gram(G):
    means = G.mean(0, keepdim=False)
    means -= means.mean() / 2
    return G - means[None, :] - means[:, None]

In [28]:
def compute_correlations(self):
    # Set `limit`
    n_words = next(iter(self.representations_d.values())).size()[0]
    if type(self.limit) == float:
        limit = int(n_words * self.limit)
    elif type(self.limit) == int:
        limit = self.limit
    else:
        limit = self.limit
    
    # Set `self.similarities`
    # {network: {other: rbfcka_similarity}}
    self.similarities = {network: {} for network in self.representations_d}
    for network, other_network in tqdm(p(self.representations_d,
                                             self.representations_d),
                                           desc='rbfcka',
                                           total=len(self.representations_d)**2):

        if network == other_network:
            continue

        if other_network in self.similarities[network]: 
            continue

        # TO DO: random subset of data using limit?
        Gx = center_gram(gram_rbf(self.representations_d[network][:limit]))
        Gy = center_gram(gram_rbf(self.representations_d[other_network][:limit]))

        scaled_hsic = torch.dot(Gx.view(-1), Gy.view(-1)).item()
        norm_gx = torch.norm(Gx, p="fro").item()
        norm_gy = torch.norm(Gy, p="fro").item()

        sim = scaled_hsic / (norm_gx*norm_gy)
        self.similarities[network][other_network] = sim
        self.similarities[other_network][network] = sim

In [30]:
# nice
compute_correlations(self)
self.similarities

rbfcka: 100%|█| 9/9 [00:00<00:00, 52.11it/s]


{'foo': {'bar': 0.13177755107494116, 'baz': 0.12516603324836942},
 'bar': {'foo': 0.13177755107494116, 'baz': 0.11312672037165297},
 'baz': {'foo': 0.12516603324836942, 'bar': 0.11312672037165297}}